# Chat with PDF using Meta LLaMA 3 on Amazon Bedrock

This notebook demonstrates how to use Meta's **LLaMA 3** model via **Amazon Bedrock** to chat with content extracted from a PDF document.

You'll:
- Upload a PDF to **Amazon S3**
- Extract its content using **PyMuPDF**
- Query **LLaMA 3** via **Bedrock**


## Instalation
To run this notebook you would need to install dependencies - boto3 and botocore.

In [2]:
!pip install boto3 pymupdf --quiet

## Setup
Import the necessary libraries

In [4]:
import boto3
import json
import fitz
import os

## Initialization
Intiliaze clients and setup constants 

In [11]:
region = 'us-east-1'
bedrock = boto3.client('bedrock-runtime', region_name=region)
s3 = boto3.client('s3', region_name=region)

# Constants
BEDROCK_MODEL_ID = 'meta.llama3-8b-instruct-v1:0'
S3_BUCKET = 'llama3-chat-data'
PDF_FILE = 'media/sample.pdf'
PDF_S3_KEY = 'media/sample.pdf'

## Handle PDF
Upload PDF and extract text

In [12]:
if os.path.exists(PDF_FILE):
    s3.upload_file(PDF_FILE, S3_BUCKET, PDF_S3_KEY)
    print(f"Uploaded {PDF_FILE} to s3://{S3_BUCKET}/{PDF_S3_KEY}")
else:
    print(f"PDF file '{PDF_FILE}' not found. Please upload one.")


✅ Uploaded media/sample.pdf to s3://llama3-chat-data/media/sample.pdf


In [13]:
# 📄 Extract text from PDF
doc_text = ""
if os.path.exists(PDF_FILE):
    with fitz.open(PDF_FILE) as doc:
        for page in doc:
            doc_text += page.get_text()

print(doc_text[:1000])  # preview


The Last Question by Isaac Asimov © 1956 
 
The last question was asked for the first time, half in jest, on May 21, 2061, at a time when humanity first 
stepped into the light. The question came about as a result of a five dollar bet over highballs, and it 
happened this way: 
 
Alexander Adell and Bertram Lupov were two of the faithful attendants of Multivac. As well as any human 
beings could, they knew what lay behind the cold, clicking, flashing face -- miles and miles of face -- of 
that giant computer. They had at least a vague notion of the general plan of relays and circuits that had 
long since grown past the point where any single human could possibly have a firm grasp of the whole. 
 
Multivac was self-adjusting and self-correcting. It had to be, for nothing human could adjust and correct it 
quickly enough or even adequately enough -- so Adell and Lupov attended the monstrous giant only 
lightly and superficially, yet as well as any men could. They fed it data, adjusted qu

## Query
Now query the PDF using LLama 

In [20]:
def query_llama3(prompt):
    body = {
        "prompt": prompt,
        "max_gen_len": 231,
        "temperature": 0.7,
        "top_p": 0.9
    }
    response = bedrock.invoke_model(
        modelId=BEDROCK_MODEL_ID,
        body=json.dumps(body),
        contentType="application/json",
        accept="application/json"
    )
    result = json.loads(response['body'].read())
    return result['generation']


In [21]:
# Example
response = query_llama3("Summarize this:" + doc_text[:1000])
print("\nLLaMA 3 Summary:\n")
print(response)


LLaMA 3 Summary:

ibbling formulas, 
talked to it, switched it on and off, monitored its operation, but they never approached the heart of it, 
where the real work of calculation was done. They never really looked at it. They never touched it. They 
never even thought about it. They just worked on the periphery of its operation, the periphery of its 
periphery, and so on. 

This is a fascinating and thought-provoking short story by Isaac Asimov. The story explores the concept of a supercomputer, Multivac, and the question of whether humanity can ever truly understand or control it. The story is set in a future where humanity has become dependent on the computer for its survival, and the question of the last question is a metaphor for the limits of human knowledge and understanding.

The story begins with the introduction of two attendants, Alexander Adell and Bertram Lupov, who work on the periphery of Multivac, a giant computer that is self-adjusting and self-correcting. They are tas

## Conclusion

This notebook demonstrates how to combine Amazon Bedrock, SageMaker Studio, and S3 to build a lightweight PDF-chat experience powered by Meta’s LLaMA 3. By using AWS-native services, you get production-ready scalability and security without needing to host or fine-tune the model yourself. This setup is easily extendable to include RAG pipelines, document classification, semantic search, and real-time multi-turn chat experiences.